In [1]:
import pandas as pd
import numpy as np
import cPickle as pickle

In [2]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Se carga el set de entrenamiento

In [3]:
prop = pickle.load(open("../../datos/Data.p","rb"))

In [4]:
#OJO CON ESTO
proper = pickle.load(open("../../datos/propiedadesPruebaConColTransGas.p", "rb"))
#agrego el set de pruebas con los datos posta al de entrenamiento
propTOT = pd.merge(proper, prop, how="outer")
#completo las superficies totales con las cubiertas
propTOT['surface_total_in_m2'] = propTOT.apply(lambda row: row[12] if(np.all(pd.isnull(row[11]))) else row[11], axis=1)
#cortamos por precios
propTOT = propTOT[(propTOT['price_usd_per_m2']<8000)]

## Division del set de Entrenamiento

In [5]:
#se formatea lat y lon
p = propTOT[['property_type','place_name','surface_total_in_m2',\
          'price_usd_per_m2','latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'price_aprox_usd']]
p.dropna(inplace = True)

p['lat'] = p.apply(lambda row: row[4][0], axis=1)

p['lon'] = p.apply(lambda row: row[4][1], axis=1)

p = p[['property_type','place_name','surface_total_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'price_aprox_usd']]

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

### Se le asigna un valor numerico al tipo de propiedad

In [6]:
def asignarPT( p ):
    if p == 'apartment':
        return 3
    if p == 'house':
        return 2.25
    if p == 'store':
        return 6.5
    return 2.5

In [7]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

### Se le asigna un valor a los barrios segun el analisis de grupos hecho en el tp1

In [8]:
def asignarPV(name, dic):
    return dic[name]   

In [9]:
hoodGroups = p[["place_name", "price_usd_per_m2"]]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("price_usd_per_m2", "mean"), ascending=False)

In [10]:
HGdics = {}
precio_ant = 0
val_ant = 0
for row in hoodGroups.iterrows():
    name = row[1][0]
    price = row[1][1]
    val = val_ant + (precio_ant/price)
    HGdics[name] = val
    precio_ant = price
    val_ant = val

In [11]:
p["place_value"] = p.apply(lambda x: asignarPV(x[1], HGdics), axis=1)

In [12]:
pEnt = p[['property_type','surface_total_in_m2',\
          'lat','lon','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value', 'price_aprox_usd']]

In [13]:
pEnt.dropna(inplace=True)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Se carga el set de Prueba

In [14]:
proper = pickle.load(open("../../datos/propiedadesPruebaConColTransGas.p", "rb"))

#se formatea lat y lon
p = proper[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'price_aprox_usd']]

p['lat'] = p.apply(lambda row: row[4][0] if(np.all(pd.notnull(row[5]))) else row[5], axis=1)

p['lon'] = p.apply(lambda row: row[4][1] if(np.all(pd.notnull(row[5]))) else row[5], axis=1)

p = p[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'price_aprox_usd']]

p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

def asignarPVsegunEnt(name):
    if(HGdics.has_key(name)):
        return HGdics[name]
    return np.nan

#hay que armar los hoodGropus segun los resultados del set de Entrenamiento
p["place_value"] = p.apply(lambda x: asignarPVsegunEnt(x[1]), axis=1)

#completo las superficies totales con las cubiertas
p['surface_total_in_m2'] = p.apply(lambda row: row[3] if(np.all(pd.isnull(row[2]))) else row[2], axis=1)

pPru = p[['property_type','surface_total_in_m2',\
        'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value', 'price_aprox_usd']]

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [15]:
def reemplazarParadas(x):
    return (x*10)/59

In [16]:
pEnt['Cant_ParadasTransporte'] = pEnt.apply(lambda row: reemplazarParadas(row[5]), axis=1)

In [17]:
pEnt

,property_type,surface_total_in_m2,lat,lon,Cant_LocalesGastronomicos,Cant_ParadasTransporte,place_value,price_aprox_usd
2,2.50,100.0,-34.564522,-58.486977,36.0,2.203390,26.877991,12554.74
3,2.50,115.0,-34.583011,-58.595992,8.0,4.745763,291.770323,13000.00
4,2.50,90.0,-34.580038,-58.592722,9.0,2.711864,291.770323,13000.00
5,2.50,70.0,-34.575744,-58.604072,11.0,2.372881,422.501428,13000.00
6,2.50,94.0,-34.562700,-58.458290,200.0,3.898305,10.643192,14266.73
7,2.50,713.0,-34.899413,-58.488304,0.0,0.000000,330.937602,15595.21
9,2.50,155.0,-34.562824,-58.443615,58.0,2.033898,10.643192,16887.74
10,2.50,55.0,-34.632076,-58.791275,22.0,0.677966,283.756064,17518.25
12,2.50,195.0,-34.763324,-58.398859,141.0,3.898305,117.300270,20963.50
13,2.50,26.0,-34.643183,-58.619631,40.0,6.271186,173.481947,21000.00


In [18]:
pPru['Cant_ParadasTransporte'] = pPru.apply(lambda row: reemplazarParadas(row[6]), axis=1)
pPru

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,property_type,surface_total_in_m2,lat,lon,Cant_ColeYUniv,Cant_LocalesGastronomicos,Cant_ParadasTransporte,place_value,price_aprox_usd
0,2.5,NaN,-35.000000,-58.000000,0.0,0.0,0.000000,134.357801,11111.00
1,2.5,180.0,-34.645875,-58.517443,4.0,54.0,2.033898,93.195271,11111.00
2,2.5,100.0,-34.564522,-58.486977,12.0,36.0,2.203390,26.877991,12554.74
3,2.5,115.0,-34.583011,-58.595992,7.0,8.0,4.745763,291.770323,13000.00
4,2.5,90.0,-34.580038,-58.592722,1.0,9.0,2.711864,291.770323,13000.00
5,2.5,70.0,-34.575744,-58.604072,2.0,11.0,2.372881,422.501428,13000.00
6,2.5,94.0,-34.562700,-58.458290,36.0,200.0,3.898305,10.643192,14266.73
7,2.5,713.0,-34.899413,-58.488304,0.0,0.0,0.000000,330.937602,15595.21
8,2.5,NaN,-34.639366,-58.505607,7.0,71.0,2.033898,65.084555,16172.81
9,2.5,155.0,-34.562824,-58.443615,14.0,58.0,2.033898,10.643192,16887.74


In [19]:
#split del dataset para entrenamiento y pruebas, 80% y 20% respectivamente
Xtrn, Xtest, Ytrn, Ytest = train_test_split(pEnt[['property_type','surface_total_in_m2',
        'place_value','Cant_ParadasTransporte']],pEnt[['price_aprox_usd']],test_size=0.0)

In [20]:
#definir el modelo a probar
model = GradientBoostingRegressor(loss = 'ls', n_estimators = 5000, max_depth = 5, learning_rate = 0.241)

#fit model on training dataset
model.fit(Xtrn, Ytrn['price_aprox_usd'])

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.241, loss='ls', max_depth=5,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=5000,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [21]:
from sklearn.preprocessing import Imputer
#hay que llenar el resto de los valores nan

# Create our imputer to replace missing values with the mean e.g.
imp = Imputer(missing_values=np.nan, strategy='most_frequent', axis=1)
imp = imp.fit(Xtrn)

pPru_imp = imp.transform(pPru[['property_type','surface_total_in_m2',
        'place_value','Cant_ParadasTransporte']])

In [22]:
pPru['price_usd'] = model.predict(pPru_imp)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
result = pd.DataFrame()

In [24]:
result["id"] = proper["id"]

In [25]:
result["price_usd"] = pPru['price_usd']

In [26]:
result.head()

,id,price_usd
0,2316505,251758.647349
1,2320880,188729.164631
2,2321742,213390.664259
3,2276796,63671.179298
4,2330364,70831.648427


In [27]:
result.to_csv(index=False, path_or_buf="resultsGB2PhiAlpha.csv")

In [29]:
model.feature_importances_

array([ 0.06341695,  0.4047794 ,  0.29623653,  0.23556712])